<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive

In [58]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
Data_basic = pd.DataFrame()

In [104]:
Data_basic = pd.DataFrame(columns=['GameID','season','types','week','HomeTeam_ID','AwayTeam_ID','Winner'])
week = list(range(1, 19))
seasons=list(range(2021,2023))

In [105]:
seasons

[2021, 2022]

In [106]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [108]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [109]:
Data_basic.to_csv('/content/drive/MyDrive/play_ID_data.csv', index=False)


In [111]:
print(Data_basic)





        GameID  season  types  week HomeTeam_ID AwayTeam_ID Winner
0    401326322    2021      2     1          27           6   Home
1    401326315    2021      2     1           1          21   Away
2    401326308    2021      2     1           2          23   Away
3    401326316    2021      2     1           4          16   Home
4    401326317    2021      2     1           8          25   Away
..         ...     ...    ...   ...         ...         ...    ...
539  401437957    2022      2    18          21          19   Home
540  401437962    2022      2    18          25          22   Home
541  401437963    2022      2    18          26          14   Home
542  401437959    2022      2    18          28           6   Home
543  401437952    2022      2    18           9           8   Away

[544 rows x 7 columns]


In [78]:
Hometeam=Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']

In [79]:
print(Hometeam.values[0])

29


In [211]:
stats_df = pd.DataFrame()
stats_df = pd.DataFrame(columns=['GameID','Pass_Incompletion_Home',
  'Punt_Home',
  'Timeout_Home',
  'Punt_Away',
  'Pass_Incompletion_Away',
  'Timeout_Away',
  'homeScore',
  'awayScore',
  'PassReception_Home_yards',
  'PassReception_Away_yards',
  'Penalty_Home',
  'Penalty_Away',
  'Penalty_Home_yards',
  'Penalty_Away_yards',
  'rushing_yards_home',
  'rushing_yards_away',
  'field_goals_good_home',
  'field_goals_good_away',
  'field_goals_missed_home',
  'field_goals_missed_away',
  'fumble_recovery_oponnent_home',
  'fumble_recovery_oponnent_away',
  'fumble_recovery_own_home',
  'fumble_recovery_own_away',
  'sack_yards_home',
  'sack_yards_away',
  'passing_TD_home',
  'passing_TD_away',
  'rushing_TD_home',
  'rushing_TD_away',
  'interception_home',
  'interception_away',
  'Interception_Return_TD_home',
  'Interception_Return_TD_away',
  'kick_off_return_TD_home',
  'kick_off_return_TD_away',
  'safety_home',
  'safety_away'])

In [206]:
def stats(GameID,Quarter):
  url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
  response_drive = r.get(url)
  drive_data = response_drive.json()
  drive_df = json_normalize(drive_data['items'])
  data=[]
  Pass_Incompletion_Home=0
  Punt_Home=0
  Timeout_Home=0
  Punt_Away=0
  Pass_Incompletion_Away=0
  Timeout_Away=0
  homeScore=0
  awayScore=0
  PassReception_Home_yards=0
  PassReception_Away_yards=0
  Penalty_Home=0
  Penalty_Away=0
  Penalty_Home_yards=0
  Penalty_Away_yards=0
  rushing_yards_home=0
  rushing_yards_away=0
  field_goals_good_home=0
  field_goals_good_away=0
  field_goals_missed_home=0
  field_goals_missed_away=0
  fumble_recovery_oponnent_home=0
  fumble_recovery_oponnent_away=0
  fumble_recovery_own_home=0
  fumble_recovery_own_away=0
  sack_yards_home=0
  sack_yards_away=0
  passing_TD_home=0
  passing_TD_away=0
  rushing_TD_home=0
  rushing_TD_away=0
  interception_home=0
  interception_away=0
  Interception_Return_TD_home=0
  Interception_Return_TD_away=0
  kick_off_return_TD_home=0
  kick_off_return_TD_away=0
  safety_home=0
  safety_away=0
  for index, row in drive_df.iterrows():
    try:
      if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
            Pass_Incompletion_Home=Pass_Incompletion_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
            Punt_Home=Punt_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
            Timeout_Home=Timeout_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
            PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
            Penalty_Home_yards=Penalty_Home_yards+row['plays.items'][detail]['statYardage']
            Penalty_Home=Penalty_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
            rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
            sack_yards_home=row['plays.items'][detail]['statYardage']+sack_yards_home
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
            passing_TD_home=passing_TD_home+1
            PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
            rushing_TD_home=rushing_TD_home+1
            rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
            interception_home=interception_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
            field_goals_good_home=field_goals_good_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
            field_goals_missed_home=field_goals_missed_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
            Interception_Return_TD_home=Interception_Return_TD_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
            kick_off_return_TD_home=kick_off_return_TD_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
            fumble_recovery_own_home=fumble_recovery_own_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
            fumble_recovery_oponnent_home=fumble_recovery_oponnent_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
            safety_home=safety_home+1
          homeScore=row['plays.items'][detail]['homeScore']
      elif (re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
            Pass_Incompletion_Away=Pass_Incompletion_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
            Punt_Away=Punt_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
            Timeout_Away=Timeout_Away+1  
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
            PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
            Penalty_Away_yards=Penalty_Away_yards+row['plays.items'][detail]['statYardage']
            Penalty_Away=Penalty_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
            rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
            #print(row['plays.items'][detail]['id'])
            #print(row['plays.items'][detail]['statYardage'])
            sack_yards_away=row['plays.items'][detail]['statYardage']+sack_yards_away
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
            passing_TD_away=passing_TD_away+1
            PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
            rushing_TD_away=rushing_TD_away+1
            rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
            interception_away=interception_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
            field_goals_good_away=field_goals_good_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
            field_goals_missed_away=field_goals_missed_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
            Interception_Return_TD_away=Interception_Return_TD_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
            kick_off_return_TD_away=kick_off_return_TD_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
            fumble_recovery_own_away=fumble_recovery_own_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
            fumble_recovery_oponnent_away=fumble_recovery_oponnent_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
            safety_away=safety_away+1
          awayScore=row['plays.items'][detail]['awayScore']
    except TypeError:
      print('it doenst worked')
  data.append(GameID)
  data.append(Pass_Incompletion_Home)
  data.append(Punt_Home)
  data.append(Timeout_Home)
  data.append(Punt_Away)
  data.append(Pass_Incompletion_Away)
  data.append(Timeout_Away)
  data.append(homeScore)
  data.append(awayScore)
  data.append(PassReception_Home_yards)
  data.append(PassReception_Away_yards)
  data.append(Penalty_Home)
  data.append(Penalty_Away)
  data.append(Penalty_Home_yards)
  data.append(Penalty_Away_yards)
  data.append(rushing_yards_home)
  data.append(rushing_yards_away)
  data.append(field_goals_good_home)
  data.append(field_goals_good_away)
  data.append(field_goals_missed_home)
  data.append(field_goals_missed_away)
  data.append(fumble_recovery_oponnent_home)
  data.append(fumble_recovery_oponnent_away)
  data.append(fumble_recovery_own_home)
  data.append(fumble_recovery_own_away)
  data.append(sack_yards_home)
  data.append(sack_yards_away)
  data.append(passing_TD_home)
  data.append(passing_TD_away)
  data.append(rushing_TD_home)
  data.append(rushing_TD_away)
  data.append(interception_home)
  data.append(interception_away)
  data.append(Interception_Return_TD_home)
  data.append(Interception_Return_TD_away)
  data.append(kick_off_return_TD_home)
  data.append(kick_off_return_TD_away)
  data.append(safety_home)
  data.append(safety_away)
  stats_df.loc[len(stats_df)] = data




In [ ]:
drive_df['plays.items'][3][2]['type']['text']

In [200]:
GameID='320909012'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
homescore=0
awayScore=0
Pass_Incompletion_Home=0
Punt_Home=0
Time_Out=0

awayScore=0
for plays in range(len(drive_df['plays.items'])):
  for detail in range(len(drive_df['plays.items'][plays])):
    if drive_df['plays.items'][plays][detail]['type']['text'] == 'Pass Incompletion':
      Pass_Incompletion_Home=Pass_Incompletion_Home+1
    elif drive_df['plays.items'][plays][detail]['type']['text'] == 'Punt':
      Punt_Home=Punt_Home+1
    elif drive_df['plays.items'][plays][detail]['type']['text'] == 'Timeout':
      Time_Out=Time_Out+1
    print(drive_df['plays.items'][plays][detail]['type']['text'])
    #print(drive_df['plays.items'][plays][detail]['statYardage'])
    #print(drive_df['plays.items'][plays][detail]['scoreValue'])
    #print(drive_df['plays.items'][plays][detail]['awayScore'])
    #print(drive_df['plays.items'][plays][detail]['homeScore'])
print(Time_Out)

Rush
Rush
Pass
Pass
Pass
Rush
Pass
Pass
Pass
Rush
Pass
Pass
Extra Point Good
Kickoff
Rush
Pass
Rush
Pass
Pass
Rush
Pass
Pass
Rush
Pass
Field Goal Good
Kickoff
Rush
Rush
Pass
Field Goal Good
Kickoff
Rush
Rush
Pass
End Period
Rush
Sack
Pass
Pass
Rush
Pass
Extra Point Good
Kickoff
Rush
Pass
Pass
Rush
Pass
Pass
Pass
Pass
Extra Point Good
Kickoff
Rush
Rush
Pass
Pass
Rush
Rush
Rush
Rush
Extra Point Good
Kickoff
Pass
Pass
Pass
Pass
Rush
Rush
Pass
Sack
Timeout
Pass
Timeout
Pass
Field Goal Good
Kickoff
Rush
End of Half
Kickoff
Rush
Rush
Rush
Rush
Pass
Rush
Pass
Rush
Pass
Pass
Field Goal Missed
Pass
Rush
Pass
Rush
Pass
Rush
Rush
Pass
Rush
Extra Point Good
Kickoff
Rush
Pass
Fumble Recovery (Opponent)
Pass
Extra Point Good
Kickoff
Sack
Rush
Pass
Rush
Pass Interception
Pass
Rush
End Period
Pass
Pass
Field Goal Good
Kickoff
Rush
Rush
Timeout
Pass Interception
Rush
Pass
Rush
Field Goal Good
Kickoff
Rush
Rush
Pass
Pass
Rush
Pass
Rush
Pass
Pass
Punt
Rush
Rush
Rush
Rush
Rush
Punt
Rush
Pass
Pass
Pass
Pas

In [ ]:
print(Pass_Incompletion_Home)

In [92]:
drive_df

,$ref,id,description,sequenceNumber,yards,isScore,offensivePlays,result,shortDisplayResult,displayResult,...,timeElapsed.value,timeElapsed.displayValue,source.id,source.description,plays.$ref,plays.count,plays.pageIndex,plays.pageSize,plays.pageCount,plays.items
0,http://sports.core.api.espn.com/v2/sports/foot...,4013263091,"6 plays, -3 yards, 3:36",1,-3,False,6,PUNT,PUNT,Punt,...,216.0,3:36,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,9,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
1,http://sports.core.api.espn.com/v2/sports/foot...,4013263092,"4 plays, 16 yards, 2:26",2,16,False,4,PUNT,PUNT,Punt,...,146.0,2:26,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,6,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
2,http://sports.core.api.espn.com/v2/sports/foot...,4013263093,"3 plays, -13 yards, 1:20",3,-13,False,3,PUNT,PUNT,Punt,...,80.0,1:20,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,6,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
3,http://sports.core.api.espn.com/v2/sports/foot...,4013263094,"4 plays, 17 yards, 3:01",4,17,False,4,PUNT,PUNT,Punt,...,181.0,3:01,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,5,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
4,http://sports.core.api.espn.com/v2/sports/foot...,4013263095,"6 plays, 5 yards, 3:27",5,5,False,6,PUNT,PUNT,Punt,...,207.0,3:27,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,8,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
5,http://sports.core.api.espn.com/v2/sports/foot...,4013263096,"11 plays, 76 yards, 4:40",6,76,False,11,FUMBLE,FUMBLE,Fumble,...,280.0,4:40,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,12,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
6,http://sports.core.api.espn.com/v2/sports/foot...,4013263097,"4 plays, 42 yards, 2:04",7,42,False,4,INT,INT,Interception,...,124.0,2:04,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,5,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
7,http://sports.core.api.espn.com/v2/sports/foot...,4013263098,"6 plays, 29 yards, 2:00",8,29,True,6,FG,FG,Field Goal,...,120.0,2:00,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,6,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
8,http://sports.core.api.espn.com/v2/sports/foot...,4013263099,"7 plays, 32 yards, 3:12",9,32,False,7,DOWNS,DOWNS,Downs,...,192.0,3:12,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,9,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
9,http://sports.core.api.espn.com/v2/sports/foot...,40132630910,"1 play, 57 yards, 0:08",10,57,True,1,TD,TD,Touchdown,...,8.0,0:08,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,1,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...


In [212]:
for plays in Data_basic['GameID']:
  stats(plays,2)
  

it doenst worked
it doenst worked


In [138]:
Data_basic[Data_basic['GameID'] == '401326508']['GameID'].values[0]

'401326508'

In [209]:
stats_df.to_csv('/content/drive/MyDrive/quarter2_data.csv', index=False)

In [189]:
stats_df

,GameID,Pass_Incompletion_Home,Punt_Home,Timeout_Home,Punt_Away,Pass_Incompletion_Away,Timeout_Away,homeScore,awayScore,PassReception_Home_yards,...,passing_TD_home,passing_TD_away,rushing_TD_home,rushing_TD_away,interception_home,interception_away,Interception_Return_TD_home,Interception_Return_TD_away,kick_off_return_TD_home,kick_off_return_TD_away
0,401326322,5,0,1,1,4,3,31,29,104,...,0,0,0,0,0,0,0,0,0,0
1,401326315,5,1,0,2,0,2,6,32,48,...,0,1,0,0,0,0,0,0,0,0
2,401326308,6,0,1,0,4,3,16,23,75,...,0,1,0,0,0,0,0,0,0,0
3,401326316,1,2,2,1,6,3,24,24,18,...,0,0,0,1,0,0,0,0,0,0
4,401326317,11,0,1,1,2,2,33,41,178,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,401437957,3,0,2,0,3,2,22,16,23,...,0,1,0,1,0,0,0,0,0,0
540,401437962,0,2,0,1,0,1,38,13,0,...,0,0,0,0,0,25,0,0,0,0
541,401437963,3,2,6,2,4,0,16,16,64,...,0,0,0,0,0,0,0,0,0,0
542,401437959,1,1,0,0,1,0,23,6,0,...,0,0,0,0,0,0,0,0,0,0
